# Qual materia os alunos faltam mais 

In [171]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, inspect 

In [172]:
engine = create_engine('sqlite:///:memory:') # Criando banco SQL

In [173]:
tabela_notas = pd.read_excel('files/tabela_nota.xlsx') # Importando tabela de notas
tabela_notas.drop(tabela_notas.loc[tabela_notas['id_materia']==0].index, inplace=True) # Não é possivel saber qual materia ele fez
tabela_notas['id'] = range(tabela_notas.shape[0])
tabela_notas.set_index('id', inplace=True)
tabela_notas.index = tabela_notas.index + 1
tabela_notas.rename(columns={'id_professor(a)': 'id_professor'}, inplace=True)
tabela_notas.to_sql('notas', engine) 
inspector = inspect(engine) 
print(inspector.get_table_names())

['notas']


In [174]:
# frequencia relativa
total_de_faltas = sum(tabela_notas['faltas'])
total_de_faltas

291

In [175]:
# importa tabela de materias
tabela_materias = pd.read_excel('files/tabela_materia.xlsx', index_col=False) # Importando tabela de professores
tabela_materias.set_index('Unnamed: 0', inplace=True)
tabela_materias.index.rename('id', inplace=True)

In [176]:
def calcula_media_faltas_por_materia(id_materia: int) -> float: 
    
    total_faltas = sum(tabela_notas[tabela_notas['id_materia'] == id_materia]['faltas'])
    total_vezes_materia_dada = 0
    trimestres = list(tabela_notas['data_inicio_trimestre'].drop_duplicates())

    for i in range(len(trimestres)):
        trimestre_busca = tabela_notas[tabela_notas['data_inicio_trimestre'] == trimestres[i]]
        trimestre_busca = trimestre_busca[trimestre_busca['id_materia'] == id_materia]

        if trimestre_busca.shape[0] == True:
            total_vezes_materia_dada += 1

    media_faltas = total_faltas/total_vezes_materia_dada
    return media_faltas

# Exportar no excel
def exporta_para_excel(itens: list, nome_coluna_itens: str, valores: list, nome_coluna_valores: str, caminho: str):
    dados = pd.DataFrame({nome_coluna_itens: itens, nome_coluna_valores: valores})
    dados.to_excel(caminho, index=False)
    
# busca lista de todas as materias
materias = pd.read_excel('files/tabela_materia.xlsx')['descricao']
materias = list(materias)

In [177]:
media_falta_materias = []
qtd_materias = tabela_materias.shape[0] + 1 

for i in range(1, qtd_materias):
    media_falta_materias.append(calcula_media_faltas_por_materia(i))
    
media_falta_materias

[7.666666666666667,
 6.75,
 8.0,
 5.6,
 6.4,
 7.0,
 6.666666666666667,
 7.75,
 4.25,
 5.5]

In [178]:
exporta_para_excel(materias, 'Materias', media_falta_materias, 'Médias de Faltas', 'files/files_questions/question_7/media_de_faltas_por_materia.xlsx')